<font color='blue'>Привет! Поздравляю с первым выполненным проектом. Проделана большая работа. Меня зовут Алексей Алексеев, я проверю твою работу и напишу свои комментарии. Они, в зависимости от результата проверки, будут в </font><font color='green'>зеленом (когда все отлично),</font> <font color='brown'>коричневом (когда можно лучше)</font><font color='blue'> или</font> <font color='red'>красном (когда нужно переделать, чтобы я принял работу).</font><font color='blue'> Ну и в синем будут просто советы и нейтральные комментарии. Пожалуйста, не удаляй эти комментарии, они понадобятся нам для дальнейшей работы над проектом. Кстати, сразу предлагаю общаться на "ты".  
Наличие критичного или трех и более некритичных замечаний является поводом для отправки проекта на доработку. Некритичные замечания могут касаться нового материала, не освещенного в тренажере, но пожалуйста, не воспринимай их как мое необоснованное желание указать на твои ошибки. Изучать новый материал по ходу работы над проектом - это часть процесса обучения, на мой взгляд, очень эффективная. Так что не стесняйся задавать вопросы, если что-то не понятно. Наша общая цель - получить новые знания и навыки.     
А чтобы я точно не пропустил твои апдейты, желательно их также подсветить другим цветом, например, <font color='purple'>фиолетовым</font>.

---   
<font color='blue'>Апдейт после проверки: отличный проект, доработать по мелочи и можно принимать.   
    Более подробно см. блок в конце.

</font>

## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [3]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
# Общая информация о таблице
print(data.info())
# Количество уникальных значений в столбце 'children'
print(data['children'].value_counts())
# Диапазон значений по столбцу 'days_employed'
print(data.sort_values(by='days_employed', ascending = False))
# Подсчет нулевых значений в столбце 'days_employed'
print(data[data['days_employed'].isnull()].count())
# Подсчет значений возраста, выходящих за разумный пределы
print(data.loc[(data['dob_years'] <= 16) | (data['dob_years'] >= 100)].count())
# По
print(data.groupby('education').count())
print(data['education_id'].value_counts())
print(data['family_status'].value_counts())
print(data['family_status_id'].value_counts())
print(data['gender'].value_counts())
print(data['income_type'].value_counts())
print(data['debt'].value_counts())
print(data['purpose'].value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None
 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64
       children  days_employed  dob_years education  education_id  \
6954          0  401755.400475         56   среднее             1   
10006         0  401715.811749         69    высшее 

### Вывод

## Общая информация об датасете
1. Количество значений в столбцах не одинаково.В двух столбцах значений меньше, соответственно в них есть пропуски данных
2. Тип значений столбцов устраивает кроме debt - надо поставить булево значение (?)
3. Анализ столбцов
  1. children
      * Посмотрев на список уникальных значений можно сделать следующие предположения:
      * 20 детей конечно может быть, но очень вряд ли. Скорее всего случайно добавили 0. Надо заменить на 2
      * Аналогично с показателем -1, надо исправить на 1
  2. days_employed
      * Очень большой разброс данных. Есть и отрицательные числа и NaN и огромныйе числа типа 482 лет (дни /365). Есть гипотезы.
      * Отрицательные величины на самом деле положительные, у которых в начале появился "-". Надо взять их по модулю
      * Пустые значения означают или отсутствие рабочего стажа или отсутствие информации. Их 2174 штуки или 10%, что существенно. Если это отсутствие информации, то можно заполнить пропуски средним значением в зависимости от возраста. Если это отсутствие рабочего стажа, то заполнить 0. Надо узнать причину пропусков. Учитывая, что данные корелируют со столбцом дохода, большая вероятность, что данные по работе не были занесены и тогда эти строки или удалить или заполнить по среднему.
      * Большие значения стажа были введены не в днях, а в часах и их надо дополнительно разделить на 24
  3. dob_years
      * 101 запись меньше 16 лет и больше 100, а таким кредит не дают. Их всего 0,4% и их можно удалить
  4. education
      * Есть дубликаты значений, написанные в разном регистре. Надо привести к нижнему для категоризации
  5. education_id
      * Есть 5 значений, соответствующие 5 типам образавания. Вопрос насколько они корректны? Можно исходя из них заполнить столбец "education" и сравнить с тем что сделали выше
  6. family_status 
      * никаких претензий нет
  7. family_status_id
      * никаких претензий нет
  8. gender 
      * одна непонятная запись "XNA", ее можно удалить 
  9. income_type
      * 4 основных категрии, 4 - незначительных. Их можно или удалить, или свести в одну категоию "прочие"
  10. debt
      * никаких претензий нет 
  11. total_income
      * Для удобства восприятия возможно отформатировать до 0 знаков после запятой. 
  12. purpose
      * Много дубликатов, связанны, видимо, со свободной формой заполнения. Надо провести леммизацию и определить категории
      
  
  
  
  
      
      
      
      
      

<font color='green'>Данные загружены и подробно изучены, использованы подходящие методы для первичного изучения данных. Отлично подмечены основные проблемы.</font>

### Шаг 2. Предобработка данных

### Обработка пропусков

In [4]:
# Приводим в порядок столбец 'children', заменяя значения
data.loc[data['children'] == -1, 'children'] = 1
data.loc[data['children'] == 20, 'children'] = 2
# Сокращаем количество категорий в столбце 'income_type'
data.loc[data['income_type'] == 'безработный', 'income_type'] = 'прочее'
data.loc[data['income_type'] == 'предприниматель', 'income_type'] = 'прочее'
data.loc[data['income_type'] == 'в декрете', 'income_type'] = 'прочее'
data.loc[data['income_type'] == 'студент', 'income_type'] = 'прочее'
# Функция вычисления медианного дохода по категориям
def median_income_type(income_type_in):
    data.loc[data['income_type'] == income_type_in]
    median_income_type = data['total_income'].median()
    return median_income_type
# Заполнение пропусков медианным доходом по категориям
data.loc[data['income_type'] == 'сотрудник','total_income'] = data.loc[data['income_type'] == 'сотрудник','total_income'].fillna(value = median_income_type('сотрудник'))
data.loc[data['income_type'] == 'пенсионер','total_income'] = data.loc[data['income_type'] == 'пенсионер','total_income'].fillna(value = median_income_type('пенсионер'))
data.loc[data['income_type'] == 'прочее','total_income'] = data.loc[data['income_type'] == 'прочее','total_income'].fillna(value = median_income_type('прочее'))
data.loc[data['income_type'] == 'компаньон','total_income'] = data.loc[data['income_type'] == 'компаньон','total_income'].fillna(value = median_income_type('компаньон'))
data.loc[data['income_type'] == 'госслужащий','total_income'] = data.loc[data['income_type'] == 'госслужащий','total_income'].fillna(value = median_income_type('госслужащий'))



<font color='green'>Отличный ход с группировкой по типу дохода. И молодец, что заполняешь медианами, а не средним арифметическим. 

<font color='blue'>Однако обрати внимание, как много у нас повторяющегося кода. Это не очень элегантно и не очень читаемо. Гораздо лучше здесь сделать зполнение в цикле: выделяем список уникальных типов занятости (`data['income_type'].unique()`), далее, проходимся по нему циклом и на каждой итерации находим медиану/зпаолняем пропуски для текущего типа занятости.  
    Оставлю на домашнее задание.

### Вывод

### Замена типа данных

In [5]:
data['total_income'] = data['total_income'].astype('int')

### Вывод

~~<font color='red'>В целом, копейки в уровнях дохода нас не интересуют, так что тоже можно заменить на int. Это сэкономит место и время на обработку, что может быть критично, если данных много.~~ 
<font color='purple'>Меняем тип данных в столбце 'total_income' на целочисленный</font>

<b>UPD: <font color='green'>Теперь все более наглядно и компактно.

### Обработка дубликатов

In [6]:
# Приведение столбца профессий к нижнему регистру
data['education'] = data['education'].str.lower()

<S><font color='red'>Молодец, что приводишь все к одному регистру, тем не менее, дубликаты (полностью одинаковые строки) в сете остались. Нужно что-то  с ними сделать.

In [7]:
# Удаляем дубликаты
data = data.drop_duplicates().reset_index(drop=True)
print(data.duplicated().sum())

0


### Вывод

<font color='purple'>В данных дубликатов нет</font>

<b>UPD: <font color='green'>Разобрались.

### Лемматизация

In [8]:
# импорт библиотеки pymystem3
from pymystem3 import Mystem
m = Mystem()
# Функция лемматизации
def purpose_lem(row):
    lemmas = ' '.join(m.lemmatize(row))
    return lemmas
# Запись лемм в новый столбец 'purpose_lem'
data['purpose_lem'] = data['purpose'].apply(purpose_lem)
# импорт контейнера для подсчёта упоминаний слов
from collections import Counter
# вывод количества уникальных лемм 
print(Counter(' '.join(data['purpose_lem'].tolist()).split())) 

        
        
        
        

Counter({'недвижимость': 6351, 'покупка': 5897, 'жилье': 4460, 'автомобиль': 4306, 'образование': 4013, 'с': 2918, 'операция': 2604, 'свадьба': 2324, 'свой': 2230, 'на': 2222, 'строительство': 1878, 'высокий': 1374, 'получение': 1314, 'коммерческий': 1311, 'для': 1289, 'жилой': 1230, 'сделка': 941, 'дополнительный': 906, 'заниматься': 904, 'проведение': 768, 'сыграть': 765, 'сдача': 651, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'подержанный': 486, 'подержать': 478, 'приобретение': 461, 'профильный': 436})


<font color='green'>Здорово реализован автоматизированный поиск популярных лемм. За это **жирный** плюс.

### Вывод

Для категорий "Цели кредита" можно выделить:
* Недвижимость / жилье / ремонт
* Автомобиль
* Образование
* Свадьба

### Категоризация данных

In [9]:
# Функция определения категории детей
def children_category(amount):
    if amount == 0:
        return 'нет детей'
    if amount >= 3:
        return 'многодетные'
    return 'есть дети'
# Новый столбец с категориями по детям
data['children_category'] = data['children'].apply(children_category)

# Функция определения категории дохода
def total_income_category (amount):
    if amount <= 100000:
        return 'Низкий доход'
    if amount >= 200000:
        return 'Высокий доход'
    return 'Средний доход'
# Новый столбец с категориями по категориям дохода
data['total_income_category'] = data['total_income'].apply(total_income_category)

# Функция определения категории цели кредита
def purpose_catrgory (row):
    for word in row.split(' '):
        if word == 'недвижимость':
            return 'недвижимость'
        if word == 'жилье':
            return 'недвижимость'        
        if word == 'ремонт':
            return 'недвижимость'
        if word == 'автомобиль':
            return 'автомобиль' 
        if word == 'образование':
            return 'образование'  
        if word == 'свадьба':
            return 'свадьба'  
# Запись категорий в новый столбец 'purpose_catrgory'
data['purpose_catrgory'] = data['purpose_lem'].apply(purpose_catrgory)       

### Вывод

Для ответов на вопросы нам понадобятся категории в столбцах:
1. Дети
    * 0 (нет детей)
    * 1-2 (есть дети)
    * 3 и больше (многодетные)
2. Семейное положение
    * женат / замужем
    * гражданский брак
    * Не женат / не замужем
    * в разводе
    * вдовец / вдова  
3. Уровень дохода
    * Низкий доход (до 20 000)
    * Средний доход (от 20 000 до 50 000)
    * Высокий доход (более 50 000)
4. Цели
    * Недвижимость / жилье / ремонт
    * Автомобиль
    * Образование
    * Свадьба

<font color='green'>Безусловно, отлично справились с категоризацией. 

<s><font color='brown'>Но давай здесь все-таки выделим хотя бы один "словарь" для образования или семейного статуса, как нас просят в задании. После этого мы теоретически сможем вовсе удалить из основной таблицы столбец со строковым обозначением образования (или семейного статуса), что здорово сэкономит место и время обработки. Просто скопируй пару столбцов, например, 'family_status' и 'family_status_id' в новый датафрейм и удали дубликаты, как было в теории в разделе "Классификация по типу".

In [10]:
# Делаем словарь для семейного положения, в дальнейшем можем вместо текстового статуся пользоваться ID
family_dict = data[['family_status', 'family_status_id']]
family_dict = family_dict.drop_duplicates().reset_index(drop=True)
print(family_dict.head(10))

           family_status  family_status_id
0        женат / замужем                 0
1       гражданский брак                 1
2         вдовец / вдова                 2
3              в разводе                 3
4  Не женат / не замужем                 4


### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [11]:
# Подсчет долей возврата кредита по категориям детей
groupby_children_category = data.groupby('children_category')['debt'].sum() / data.groupby('children_category')['debt'].count()
print(groupby_children_category.sort_values())


children_category
нет детей      0.075438
многодетные    0.081579
есть дети      0.092654
Name: debt, dtype: float64


<font color='green'>Все верно. Отличное использование `groupby`!

### Вывод

Между наличием детей и возвратом кредита в срок зависимости практически нет. Лучше всего возвращают кредиты заемщики без детей.

<font color='blue'>Насчет отсутствия разницы. Допустим банк дает кредит под 15% годовах. Из них 5% "съедает" инфляция, 5% - операционные расходы. Итого +5% чистой прибыли. При 4% невозврата банк еще в плюсе, возможно, это миллионы доларов. При 5% невозврата - банк не зарабатывает ничнего, при 6% - он банкротится.  
    Конечно, эта ситуация крайне утрирована для примера, но то, что для бизнеса несколько процентов могут стать вопросом жизни и смерти - это факт.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [12]:
groupby_family_status = data.groupby('family_status')['debt'].sum() / data.groupby('family_status')['debt'].count()
print(groupby_family_status.sort_values())

family_status
вдовец / вдова           0.065693
в разводе                0.071130
женат / замужем          0.075452
гражданский брак         0.093471
Не женат / не замужем    0.097509
Name: debt, dtype: float64


### Вывод

Между семейным положением и возвратом кредита в срок зависимомь минимальна. Наиболее аккуратные - "вдовец / вдова", наиболее проблемные - без семьи

<font color='blue'>В целом, здесь видно, что наиболее надежные заемщики, это те, кто находится или когда-либо был в официальном браке.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [13]:
# Подсчет долей возврата кредита по категориям дохода
groupby_income_category = data.groupby('total_income_category')['debt'].sum() / data.groupby('total_income_category')['debt'].count()
print(groupby_income_category.sort_values())

total_income_category
Высокий доход    0.070667
Низкий доход     0.079319
Средний доход    0.086289
Name: debt, dtype: float64


### Вывод

Чем выше доход, тем меньше проблем с возратами кредита. Интересно было бы добавить информацию по сумме кредита. 

<font color='blue'>Однкао, что характерно, клиенты с самым нихким уровнем дохода отдают кредиты лучше, чем клиенты со средним.

- Как разные цели кредита влияют на его возврат в срок?

In [14]:
groupby_purpose = data.groupby('purpose_catrgory')['debt'].sum() / data.groupby('purpose_catrgory')['debt'].count()
print(groupby_purpose.sort_values())

purpose_catrgory
недвижимость    0.072334
свадьба         0.080034
образование     0.092200
автомобиль      0.093590
Name: debt, dtype: float64


### Вывод

Лучше всего возвращают кредиты на автомобиль, хуже всего - на недвижимость
<font color='purple'>Лучше всего возвращают кредиты на недвижимость, хуже всего - на аввтомобиль
</font>

~~<font color='red'>Перепутали числитель со знаменателем. В итоге выводы полностью противоположные верным.~~
<font color='purple'>Да, ошибся. Исправил</font>    

<font color='green'>В целом же, эта часть выполнена хорошо. Верные цифры, лаконичный код, четкие и правильные выводы.

~~<font color='red'>Однако одной из целей этой работы является отработка применения сводных таблиц. Это гибкий и удобный способ, развитие метода `groupby()`, которым, как я вижу, ты уже вполне умеешь пользоваться. Предлагаю сделать шаг вперед и применить в рассчетах метод `pivot_table()`. Подсказка: для ответов на поставленные вопросы можно не указывать аргумент `columns` - тогда мы получим одномерную табличку, как в случае с `groupby()`.~~  
  <font color='blue'> При желании же копнуть глубже, можно указать в качестве колонок (аргумент `columns`) какую-либо вторую категорию, тогда мы сможем оценить, например, как отдают кредиты люди с детьми и с целью - автомобиль.

In [15]:
print(data.pivot_table('debt', index='children_category'))
print(data.pivot_table('debt', index='family_status'))
print(data.pivot_table('debt', index='total_income_category'))
print(data.pivot_table('debt', index='purpose_catrgory'))
print(data.pivot_table('debt', index='purpose_catrgory', columns = 'family_status'))

                       debt
children_category          
есть дети          0.092654
многодетные        0.081579
нет детей          0.075438
                           debt
family_status                  
Не женат / не замужем  0.097509
в разводе              0.071130
вдовец / вдова         0.065693
гражданский брак       0.093471
женат / замужем        0.075452
                           debt
total_income_category          
Высокий доход          0.070667
Низкий доход           0.079319
Средний доход          0.086289
                      debt
purpose_catrgory          
автомобиль        0.093590
недвижимость      0.072334
образование       0.092200
свадьба           0.080034
family_status     Не женат / не замужем  в разводе  вдовец / вдова  \
purpose_catrgory                                                     
автомобиль                     0.128728   0.074733        0.091743   
недвижимость                   0.081454   0.069527        0.051661   
образование                    0.1

<b>UPD: <font color='green'>Отлично, воспользовались этим полезным инструментом на полную мощь!

### Шаг 4. Общий вывод

Анализ предоставленных данных показал:
1. Между наличием детей и возвратом кредита в срок зависимости практически нет. Лучше всего возвращают кредиты заемщики без детей.
2. Между семейным положением и возвратом кредита в срок зависимомь минимальна. Наиболее аккуратные - "вдовец / вдова", наиболее проблемные - без семьи
3. Наиболее аккуратно платят заемщики с высоким доходом, а хуже  - со средним доходом.
4. Лучше всего возвращают кредиты на автомобиль, хуже всего - на недвижимость
Рекомендации к дальнейшему сбору данных:
1. Уточнить указание к заполнении "Общего трудового стажа", например месяцы и сделать предварительную проверку на адекватность значения
2. При заполнении сделать категории типов занятости.
3. При заполнении сделать категории целей кредита.
Дополнительно:
Возможно имеет смысл рассчитать кросс-категорийные показатели. Например: сумма кредита и доход заемщика, цели кредита и семейное положение и т.д.
<font color='purple'>Дополнительный выводы:
    При цели кредита "на автомобиль" лучше всего возвращают заемщики, находящиеся в разводе, хуже всего - не женатые
    При цели кредита "на недвижимость" лучше всего возвращают заемщики в статусе "вдовец/вдова", хуже всего - в гражданском браке
    При цели кредита "на образование" лучше всего возвращают заемщики в статусе "женат/замужем", хуже всего - в гражданском браке
    Кредит "на свадьбу" берут только заемщики в гражданском браке 
</font>    


<font color=green>Очень хороший структурированный вывод. Здорово, что в ты даешь четкий ответ на центральный вопрос, то есть бизнес совет клиенту - кто для него хороший клиент, кто плохой. Именно этого он от нас обычно и ждет.

<font color=green>К оформлению работы вопросов нет: все аккуратно, код и комментарии в подходящих ячейках. Код читабелен, комментарии грамотны и понятны.

---
<font color='blue'>Итак, отличная работа, особенно для первого проекта. Ты показываешь прекрасное владение изучаемым материалом: уверенно пользуешься pandas, умеешь подготавливать, очищать, обогощать данные. Также ты правильно разбиваешь их на категории, грамотно применяя метод `apply()`. Отдельно отмечу очень лаконичный и суперчитабельный код.   
    Однако некоторые замечания у меня все же есть, уверен, учесть их тебе не составит труда. Жду доработанный проект, успехов!

<b>UPD: <font color='blue'>Работа доведена до идеала, с удовольствием принимаю. Отличный старт выполнения проектов. Впереди еще много интересных исследований. Успехов в дальнейшем прохождении курса!

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [х]  заменен вещественный тип данных на целочисленный;
- [х]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.